In [1]:
import pathlib
import sys
import shutil
import tempfile
import os
import subprocess
import csv
import biom
import numpy as np
import pandas as pd

from qiime2 import Artifact, Metadata
from qiime2.plugins import cutadapt as cut
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.dada2.methods import denoise_single
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize as summarize_table
from qiime2.plugins.metadata.visualizers import tabulate

from length_filtering import length_filter

def extract_all(file, dest):
    try:
        os.mkdir(dest)
    except:
        pass
    with tempfile.TemporaryDirectory() as temp:
        file.export_data(temp)
        temp_pathlib = pathlib.Path(temp)
        for file in temp_pathlib.iterdir():
            shutil.copy(file, dest)

In [2]:
sample_map = 'Sample_Map_Parasite.txt'

metadata = Metadata.load(sample_map)
metadata_df = pd.read_csv(sample_map, sep='\t')
metadata_df

,#SampleID,BarcodeSequence,Sample_year,Sample_Name,MORPH,SEX,PAIR_TYPE,HABITAT
0,bc1001,TCAGACGATGCGTCAT,2019,GFGB,Tan,Male,TxW,Bog
1,bc1002,CTATACATGACTCTGC,2019,FWWW,Tan,Female,WxT,Bog
2,bc1003,TACTAGAGTAGCACTC,2019,PFGR,White,Female,TxW,Pond
3,bc1004,TGTGTATCAGTACATG,2019,WKOF,Tan,Male,TxW,Forest
4,bc1005,ACACGCATGACACACT,2019,RWWF,Tan,Female,WxT,Forest
...,...,...,...,...,...,...,...,...
86,bc1091,CACACGAGATCTCATC,2007,FORW.2,Tan,Male,TxW,Forest
87,bc1092,AGATACACATGATACT,2013,GFOO,White,Male,WxT,Bog
88,bc1094,TCTCGACTGCACATAT,2011,BOFG.2,Tan,Female,TxT,Bog
89,bc1095,TGAGTGACGTGTAGCG,2011,YFOW,Tan,Male,TxW,Pond


In [5]:
mux = Artifact.import_data("MultiplexedSingleEndBarcodeInSequence", "all_seqs.fastq.gz")

mux.save('mux.qza')

In [7]:
demux, untrimmed = cut.methods.demux_single(mux, 
                                            metadata.get_column('BarcodeSequence'),
                                            error_rate = 0)

demux.save('demux.qza')

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: cutadapt --front file:/tmp/tmpm54bd7ib --error-rate 0 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-ir7of0v3/{name}.1.fastq.gz --untrimmed-output /tmp/q2-MultiplexedSingleEndBarcodeInSequenceDirFmt-keblls0t/forward.fastq.gz /tmp/qiime2-archive-g2vrwdl0/dec41ac4-0407-49b1-b4f1-073f13cf81fe/data/forward.fastq.gz

This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --front file:/tmp/tmpm54bd7ib --error-rate 0 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-ir7of0v3/{name}.1.fastq.gz --untrimmed-output /tmp/q2-MultiplexedSingleEndBarcodeInSequenceDirFmt-keblls0t/forward.fastq.gz /tmp/qiime2-archive-g2vrwdl0/dec41ac4-0407-49b1-b4f1-073f13cf81fe/data/forward.fastq.gz
Processing reads on 1 core in single-en

<visualization: Visualization uuid: c9201558-b217-4dd8-80ad-3e5e2ad8aaf5>

In [12]:
demux = Artifact.load('QIIME_Artifacts/demux.qza')
demux_viz = summarize(demux)
(demux_viz.visualization).save('QIIME_Artifacts/demux.qza')
demux_viz.visualization

<visualization: Visualization uuid: cc01ef5d-d69f-4d8f-9b85-e9cd2ae49562>

In [8]:
demux = Artifact.load('QIIME_Artifacts/demux.qza')

In [13]:
#Universal Primer Trimming WITHOUT minimum Length

adapters = ["GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA",
 "TGGATCACTTGTGCAAGCATCACATCGTAGCATTATCTGGATGAGATAATGGNGC...AATGTAAGTATCTAAAACACCATGTGACCTGAGTCAGCTACATGTTCGACTGC",
 "GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGCTTTCGATATATGCATG...ACCATTATCACATCCAGATAATGCCTACGATGTGATGCTTGCACAAGTGATCCA",
 "TGGATCACTTGTGCAAGCATCACATCGTAGGCATTATCTGGATGTGATAATGGT...CATGCATATATCGAAAGCACCATGTGACCTGAGTCAGCTACATGTTCGACTGC"]

trimmed_no_min = cut.methods.trim_single(demux,
                                  cores=34,
                                  adapter = adapters,
                                  error_rate = .2,
                                  indels = True,
                                  discard_untrimmed = True)

trimmed_no_min_sum = summarize(trimmed_no_min.trimmed_sequences)
trimmed_no_min_sum.visualization

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 34 --error-rate 0.2 --times 1 --overlap 3 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-09iymnen/bc1001_TCAGACGATGCGTCAT_L001_R1_001.fastq.gz --adapter GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA --adapter TGGATCACTTGTGCAAGCATCACATCGTAGCATTATCTGGATGAGATAATGGNGC...AATGTAAGTATCTAAAACACCATGTGACCTGAGTCAGCTACATGTTCGACTGC --adapter GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGCTTTCGATATATGCATG...ACCATTATCACATCCAGATAATGCCTACGATGTGATGCTTGCACAAGTGATCCA --adapter TGGATCACTTGTGCAAGCATCACATCGTAGGCATTATCTGGATGTGATAATGGT...CATGCATATATCGAAAGCACCATGTGACCTGAGTCAGCTACATGTTCGACTGC --discard-untrimmed /tmp/qiime2-archive-pcgldx7l/b51c17ce-56a9-4781-b044-69e4972a2f7f/data/bc

Finished in 2.40 s (342 µs/read; 0.18 M reads/minute).

=== Summary ===

Total reads processed:                   7,015
Reads with adapters:                     7,014 (100.0%)

== Read fate breakdown ==
Reads that were too short:                  23 (0.3%)
Reads discarded as untrimmed:                1 (0.0%)
Reads written (passing filters):         6,991 (99.7%)

Total basepairs processed:     4,118,418 bp
Total written (filtered):      3,311,961 bp (80.4%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 295 times; 3' trimmed: 292 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.39 s (510 µs/read; 0.12 M reads/minute).

=== Summary ===

Total reads processed:                   4,678
Reads with adapters:                     4,678 (100.0%)

== Read fate breakdown ==
Reads that were too short:                  16 (0.3%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         4,662 (99.7%)

Total basepairs processed:     2,706,192 bp
Total written (filtered):      2,184,176 bp (80.7%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 131 times; 3' trimmed: 131 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.41 s (231 µs/read; 0.26 M reads/minute).

=== Summary ===

Total reads processed:                  10,432
Reads with adapters:                    10,432 (100.0%)

== Read fate breakdown ==
Reads that were too short:                   6 (0.1%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):        10,426 (99.9%)

Total basepairs processed:     6,090,429 bp
Total written (filtered):      4,942,392 bp (81.2%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 369 times; 3' trimmed: 370 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.40 s (301 µs/read; 0.20 M reads/minute).

=== Summary ===

Total reads processed:                   7,969
Reads with adapters:                     7,968 (100.0%)

== Read fate breakdown ==
Reads that were too short:                  14 (0.2%)
Reads discarded as untrimmed:                1 (0.0%)
Reads written (passing filters):         7,954 (99.8%)

Total basepairs processed:     4,447,622 bp
Total written (filtered):      3,534,396 bp (79.5%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 3103 times; 3' trimmed: 3101 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 

...

Finished in 2.28 s (702 µs/read; 0.09 M reads/minute).

=== Summary ===

Total reads processed:                   3,245
Reads with adapters:                     3,245 (100.0%)

== Read fate breakdown ==
Reads that were too short:                   2 (0.1%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         3,243 (99.9%)

Total basepairs processed:     1,854,293 bp
Total written (filtered):      1,474,481 bp (79.5%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 1525 times; 3' trimmed: 1524 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 

Finished in 2.45 s (236 µs/read; 0.25 M reads/minute).

=== Summary ===

Total reads processed:                  10,386
Reads with adapters:                    10,386 (100.0%)

== Read fate breakdown ==
Reads that were too short:                  15 (0.1%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):        10,371 (99.9%)

Total basepairs processed:     5,391,195 bp
Total written (filtered):      4,248,342 bp (78.8%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 136 times; 3' trimmed: 134 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.54 s (300 µs/read; 0.20 M reads/minute).

=== Summary ===

Total reads processed:                   8,463
Reads with adapters:                     8,463 (100.0%)

== Read fate breakdown ==
Reads that were too short:                  11 (0.1%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         8,452 (99.9%)

Total basepairs processed:     4,322,766 bp
Total written (filtered):      3,356,482 bp (77.6%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 455 times; 3' trimmed: 457 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.35 s (367 µs/read; 0.16 M reads/minute).

=== Summary ===

Total reads processed:                   6,416
Reads with adapters:                     6,416 (100.0%)

== Read fate breakdown ==
Reads that were too short:                   3 (0.0%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         6,413 (100.0%)

Total basepairs processed:     3,645,194 bp
Total written (filtered):      2,940,030 bp (80.7%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 145 times; 3' trimmed: 143 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8

<visualization: Visualization uuid: 4a2e3c0a-a2fa-4e93-a6da-2fedabdd60eb>

In [12]:
#Universal Primer Trimming WITH minimum Length

adapters = ["GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA",
 "TGGATCACTTGTGCAAGCATCACATCGTAGCATTATCTGGATGAGATAATGGNGC...AATGTAAGTATCTAAAACACCATGTGACCTGAGTCAGCTACATGTTCGACTGC",
 "GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGCTTTCGATATATGCATG...ACCATTATCACATCCAGATAATGCCTACGATGTGATGCTTGCACAAGTGATCCA",
 "TGGATCACTTGTGCAAGCATCACATCGTAGGCATTATCTGGATGTGATAATGGT...CATGCATATATCGAAAGCACCATGTGACCTGAGTCAGCTACATGTTCGACTGC"]

trimmed = cut.methods.trim_single(demux,
                                  cores=34,
                                  adapter = adapters,
                                  minimum_length = 470,
                                  error_rate = .2,
                                  indels = True,
                                  discard_untrimmed = True)

(trimmed.trimmed_sequences).save('QIIME_Artifacts/trimmed.qza')

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 34 --error-rate 0.2 --times 1 --overlap 3 --minimum-length 470 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-7pc2oqwu/bc1001_TCAGACGATGCGTCAT_L001_R1_001.fastq.gz --adapter GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA --adapter TGGATCACTTGTGCAAGCATCACATCGTAGCATTATCTGGATGAGATAATGGNGC...AATGTAAGTATCTAAAACACCATGTGACCTGAGTCAGCTACATGTTCGACTGC --adapter GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGCTTTCGATATATGCATG...ACCATTATCACATCCAGATAATGCCTACGATGTGATGCTTGCACAAGTGATCCA --adapter TGGATCACTTGTGCAAGCATCACATCGTAGGCATTATCTGGATGTGATAATGGT...CATGCATATATCGAAAGCACCATGTGACCTGAGTCAGCTACATGTTCGACTGC --discard-untrimmed /tmp/qiime2-archive-dgfffe95/b51c17ce-56a9-4781-b044-69e4972a2f7f/data/

Finished in 2.40 s (342 µs/read; 0.18 M reads/minute).

=== Summary ===

Total reads processed:                   7,015
Reads with adapters:                     7,014 (100.0%)

== Read fate breakdown ==
Reads that were too short:                 598 (8.5%)
Reads discarded as untrimmed:                1 (0.0%)
Reads written (passing filters):         6,416 (91.5%)

Total basepairs processed:     4,118,418 bp
Total written (filtered):      3,095,578 bp (75.2%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 295 times; 3' trimmed: 292 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.38 s (508 µs/read; 0.12 M reads/minute).

=== Summary ===

Total reads processed:                   4,678
Reads with adapters:                     4,678 (100.0%)

== Read fate breakdown ==
Reads that were too short:                 383 (8.2%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         4,295 (91.8%)

Total basepairs processed:     2,706,192 bp
Total written (filtered):      2,058,035 bp (76.0%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 131 times; 3' trimmed: 131 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.40 s (230 µs/read; 0.26 M reads/minute).

=== Summary ===

Total reads processed:                  10,432
Reads with adapters:                    10,432 (100.0%)

== Read fate breakdown ==
Reads that were too short:                 514 (4.9%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         9,918 (95.1%)

Total basepairs processed:     6,090,429 bp
Total written (filtered):      4,745,113 bp (77.9%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 369 times; 3' trimmed: 370 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8;

Finished in 2.39 s (299 µs/read; 0.20 M reads/minute).

=== Summary ===

Total reads processed:                   7,969
Reads with adapters:                     7,968 (100.0%)

== Read fate breakdown ==
Reads that were too short:               3,720 (46.7%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         4,249 (53.3%)

Total basepairs processed:     4,447,622 bp
Total written (filtered):      2,030,476 bp (45.7%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 3103 times; 3' trimmed: 3101 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp:

...

Finished in 2.45 s (143 µs/read; 0.42 M reads/minute).

=== Summary ===

Total reads processed:                  17,181
Reads with adapters:                    17,181 (100.0%)

== Read fate breakdown ==
Reads that were too short:               9,127 (53.1%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         8,054 (46.9%)

Total basepairs processed:     9,120,811 bp
Total written (filtered):      3,951,644 bp (43.3%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 445 times; 3' trimmed: 444 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8

Finished in 2.26 s (697 µs/read; 0.09 M reads/minute).

=== Summary ===

Total reads processed:                   3,245
Reads with adapters:                     3,245 (100.0%)

== Read fate breakdown ==
Reads that were too short:               1,597 (49.2%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         1,648 (50.8%)

Total basepairs processed:     1,854,293 bp
Total written (filtered):        786,763 bp (42.4%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 1525 times; 3' trimmed: 1524 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp:

Finished in 2.43 s (234 µs/read; 0.26 M reads/minute).

=== Summary ===

Total reads processed:                  10,386
Reads with adapters:                    10,386 (100.0%)

== Read fate breakdown ==
Reads that were too short:               5,297 (51.0%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         5,089 (49.0%)

Total basepairs processed:     5,391,195 bp
Total written (filtered):      2,435,719 bp (45.2%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 136 times; 3' trimmed: 134 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8

Finished in 2.49 s (295 µs/read; 0.20 M reads/minute).

=== Summary ===

Total reads processed:                   8,463
Reads with adapters:                     8,463 (100.0%)

== Read fate breakdown ==
Reads that were too short:               8,146 (96.3%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):           317 (3.7%)

Total basepairs processed:     4,322,766 bp
Total written (filtered):        195,785 bp (4.5%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 455 times; 3' trimmed: 457 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 

Finished in 2.34 s (364 µs/read; 0.16 M reads/minute).

=== Summary ===

Total reads processed:                   6,416
Reads with adapters:                     6,416 (100.0%)

== Read fate breakdown ==
Reads that were too short:               1,394 (21.7%)
Reads discarded as untrimmed:                0 (0.0%)
Reads written (passing filters):         5,022 (78.3%)

Total basepairs processed:     3,645,194 bp
Total written (filtered):      2,485,406 bp (68.2%)

=== Adapter 3 ===

Sequence: GCAGTCGAACATGTAGCTGACTCAGGTCACATGGTGTTTTAGATACTTACATT...GCNCCATTATCTCATCCAGATAATGCTACGATGTGATGCTTGCACAAGTGATCCA; Type: linked; Length: 53+55; 5' trimmed: 145 times; 3' trimmed: 143 times

Minimum overlap: 3+3
No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8; 45-49 bp: 9; 50-53 bp: 10

No. of allowed errors:
1-4 bp: 0; 5-9 bp: 1; 10-14 bp: 2; 15-19 bp: 3; 20-24 bp: 4; 25-29 bp: 5; 30-34 bp: 6; 35-39 bp: 7; 40-44 bp: 8

In [11]:
trimmed = Artifact.load('QIIME_Artifacts/trimmed.qza')
trimmed_sum = summarize(trimmed)
trimmed_sum.visualization.save('QIIME_Artifacts/trimmed.qzv')
trimmed_sum.visualization

<visualization: Visualization uuid: d0f1b9c7-c5f1-476f-865d-48be8cf9294d>

In [15]:
table, rep_seqs, stats = denoise_single(trimmed,
                                              trunc_len = 0,
                                              n_threads = 34)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-ccuzt4je/91fd3782-69c4-4a47-a6ad-927ced89dfb3/data /tmp/tmpv6y5nz2q/output.tsv.biom /tmp/tmpv6y5nz2q/track.tsv /tmp/tmpv6y5nz2q 0 0 2.0 2 Inf independent consensus 1.0 34 1000000 NULL 16

R version 4.0.5 (2021-03-31) 


Loading required package: Rcpp


DADA2: 1.18.0 / Rcpp: 1.0.7 / RcppParallel: 5.1.4 
1) Filtering ...........................................................................................
2) Learning Error Rates
283526088 total bases in 589865 reads from 91 samples will be used for learning the error rates.
3) Denoise samples ...........................................................................................
4) Remove chimeras (method = consensus)
5) Report read numbers through the pipeline
6) Write output


In [16]:
table_sum = summarize_table(table)
tabulated_rep_seqs = tabulate_seqs(rep_seqs)
table_sum.visualization

<visualization: Visualization uuid: 286ddb3e-006a-40d8-9887-22ce2d141c13>

In [17]:
tabulated_rep_seqs.visualization

<visualization: Visualization uuid: 085c686d-5bff-4075-ac8a-08ee9878b08b>

In [18]:
rep_seqs_filt, table_filt = length_filter(rep_seqs, table, 485)

In [27]:
filt_table_sum = summarize_table(table_filt.filtered_table)
filt_tabulated_rep_seqs = tabulate_seqs(rep_seqs_filt.filtered_data)
filt_table_sum.visualization

<visualization: Visualization uuid: 77eabd5d-d21b-4421-ad94-c79eccc8b6f2>

In [28]:
filt_tabulated_rep_seqs.visualization

<visualization: Visualization uuid: fdd518e4-2dc7-4738-9b52-961e3e046930>

In [32]:
extract_all(table,'denoised_files')
extract_all(rep_seqs,'denoised_files')
extract_all(stats,'denoised_files')
extract_all(table_filt.filtered_table,'denoised_files/length_filtered')
extract_all(rep_seqs_filt.filtered_data,'denoised_files/length_filtered')
table.save('QIIME_Artifacts/table.qza')
rep_seqs.save('QIIME_Artifacts/rep_seqs.qza')
(table_filt.filtered_table).save('QIIME_Artifacts/table_filt.qza')
(rep_seqs_filt.filtered_data).save('QIIME_Artifacts/rep_seqs_filt.qza')
stats.save('QIIME_Artifacts/stats.qza')

'QIIME_Artifacts/stats.qza'

In [ ]:
#Reloading to save as QZVs

In [18]:
filt_table = Artifact.load('QIIME_Artifacts/table_filt.qza')
filt_seqs = Artifact.load('QIIME_Artifacts/rep_seqs_filt.qza')

In [19]:
filt_table_qzv = (summarize_table(filt_table)).visualization

In [20]:
filt_table_qzv

<visualization: Visualization uuid: 0d894210-287e-40f3-b079-1cf1a35d2767>

In [21]:
filt_seqs_qzv = (tabulate_seqs(filt_seqs)).visualization

In [22]:
filt_seqs_qzv

<visualization: Visualization uuid: f9d18301-f6b7-42fc-ad4d-8c6d680bd406>

In [14]:
filt_seqs_qzv.save('Visuals/seqs_filt.qzv')
filt_table_qzv.save('Visuals/table_filt.qzv')

'Visuals/table_filt.qzv'